<a href="https://colab.research.google.com/github/kyxyxn/Telemedicine/blob/main/3_CEM_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd   
from functools import reduce
from scipy.stats import entropy

In [3]:
rootPath ='/content/drive/MyDrive/ASU/CIS791_Chen_Final/uds-data/'
filenames_uds = [rootPath+'uds-2014-full-dataset.xlsx',
                 rootPath+'uds-2015-full-dataset.xlsx',
                 rootPath+'uds-2016-full-dataset.xlsx',
                 rootPath+'uds-2017-full-dataset.xlsx',
                 rootPath+'uds-2018-full-dataset.xlsx',
                 rootPath+'uds-2019-full-dataset.xlsx',
                 rootPath+'uds-2020-full-dataset.xlsx']

In [4]:
hc_df = pd.read_csv(rootPath+'samples_hc.csv')
visits_df = pd.read_csv(rootPath+'dv_visits.csv')

## Merge
hc_df['hc_df']=1
visits_df['visits_df']=1
temp_df = pd.merge(hc_df,visits_df, on=['Grant Number','year'], how='outer')
temp_df = temp_df[~temp_df['hc_df'].isna()]
df = temp_df.copy()

In [5]:
# These should be identified
grant_list = df['Grant Number'].value_counts().index.tolist()
# These are the states
state_list = df['Site State'].value_counts().index.tolist()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5833 entries, 0 to 5832
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bhcmis_id                3618 non-null   object 
 1   Grant Number             5833 non-null   object 
 2   year                     5833 non-null   int64  
 3   Health Center Name       3618 non-null   object 
 4   rural_flag_center        3618 non-null   object 
 5   use_telehealth           2612 non-null   float64
 6   mental_telehealth        2612 non-null   float64
 7   consistency_1            2612 non-null   float64
 8   consistency_2            2612 non-null   float64
 9   Site State               5833 non-null   object 
 10  hc_df                    5833 non-null   float64
 11  alcohol_num_visits       5826 non-null   float64
 12  alcohol_num_patients     5826 non-null   float64
 13  tobacco_num_visits       5826 non-null   float64
 14  tobacco_num_patients    

### HC characteristics

#### 1. Number of patients

In [7]:
# Table 4 -- Total number of patients
def get_df(filenames, table):
  final_df = pd.DataFrame()
  for i in range(len(filenames)):
    print(i)
    temp_df = pd.read_excel(filenames[i], sheet_name=table)
    temp_df.columns = temp_df.iloc[0].values.tolist()
    print(temp_df.columns.values.tolist()[:5])
    temp_df['year']=filenames[i].split('uds-')[-1][:4]
    final_df = final_df.append(temp_df).reset_index(drop=True)
  return final_df

t4_df = get_df(filenames_uds, 'Table4')
t4_df = t4_df[['GrantNumber', 'year', 'T4_L6_Ca', 'T4_L7_Ca',	'T4_L7_Cb', 'T4_L8_Ca', 'T4_L8_Cb', 'T4_L9_Ca','T4_L9_Cb', 'T4_L11_Ca', 'T4_L11_Cb']]
t4_df = t4_df[t4_df['GrantNumber']!='GrantNumber'].reset_index(drop=True)
t4_df.columns = ['Grant Number', 'year', 'total_num_patients', 'total_num_patients_uninsured_17','total_num_patients_uninsured_18older', 'total_num_patients_medicaid_17', 'total_num_patients_medicaid_18older',
                 'total_num_patients_medicare_17', 'total_num_patients_medicare_18older', 'total_num_patients_private_17', 'total_num_patients_private_18older']
t4_df = t4_df.replace(to_replace={'-':np.nan})
t4_df = t4_df[~t4_df['total_num_patients'].isna()].reset_index(drop=True)
t4_df = t4_df.fillna(0)

t4_df['total_num_patients_medicaid'] = t4_df['total_num_patients_medicaid_17']+t4_df['total_num_patients_medicaid_18older']
t4_df['total_num_patients_medicare'] = t4_df['total_num_patients_medicare_17']+t4_df['total_num_patients_medicare_18older']
t4_df['total_num_patients_private'] = t4_df['total_num_patients_private_17']+t4_df['total_num_patients_private_18older']
t4_df['total_num_patients_uninsured'] = t4_df['total_num_patients_uninsured_17']+t4_df['total_num_patients_uninsured_18older']

t4_df = t4_df[['Grant Number', 'year', 'total_num_patients', 'total_num_patients_medicaid', 'total_num_patients_medicare', 'total_num_patients_private', 'total_num_patients_uninsured']]
t4_df = t4_df[t4_df['Grant Number'].isin(grant_list)].reset_index(drop=True) # Focus

0
['BHCMISID', 'GrantNumber', 'T4_L1_Ca', 'T4_L2_Ca', 'T4_L3_Ca']
1
['BHCMISID', 'GrantNumber', 'T4_L1_Ca', 'T4_L2_Ca', 'T4_L3_Ca']
2
['BHCMISID', 'GrantNumber', 'T4_L1_Ca', 'T4_L2_Ca', 'T4_L3_Ca']
3
['BHCMISID', 'GrantNumber', 'T4_L1_Ca', 'T4_L2_Ca', 'T4_L3_Ca']
4
['BHCMISID', 'GrantNumber', 'T4_L1_Ca', 'T4_L2_Ca', 'T4_L3_Ca']
5
['BHCMISID', 'GrantNumber', 'T4_L1_Ca', 'T4_L2_Ca', 'T4_L3_Ca']
6
['BHCMISID', 'GrantNumber', 'T4_L1_Ca', 'T4_L2_Ca', 'T4_L3_Ca']


In [8]:
t4_df.info() # 5830

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5837 entries, 0 to 5836
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Grant Number                  5837 non-null   object 
 1   year                          5837 non-null   object 
 2   total_num_patients            5837 non-null   float64
 3   total_num_patients_medicaid   5837 non-null   float64
 4   total_num_patients_medicare   5837 non-null   float64
 5   total_num_patients_private    5837 non-null   float64
 6   total_num_patients_uninsured  5837 non-null   float64
dtypes: float64(5), object(2)
memory usage: 319.3+ KB


#### 2. Financial assistance

In [9]:
money_df = pd.read_excel(r'/content/drive/MyDrive/ASU/CIS791_Chen_Final/uds-data/hc_site_zips.xlsx')
money_df = money_df[~money_df['Grantee Name'].isna()]
money_df = money_df[['Grant #', 'Fiscal Year', 'Financial Assistance']].sort_values(by=['Grant #', 'Fiscal Year']).reset_index(drop=True)
money_df = money_df.rename(columns={'Grant #':'Grant Number', 'Fiscal Year':'year'})
money_df['Financial Assistance'] = money_df['Financial Assistance'].replace(to_replace={'\$':'', ',':''}, regex=True)
money_df.loc[:,'Financial Assistance'] = money_df.loc[:,'Financial Assistance'].astype(float)
money_df.loc[:,'year'] = money_df.loc[:,'year'].astype(int)
money_df = money_df[money_df['Grant Number'].isin(grant_list)].reset_index(drop=True) # Focus
money_df = money_df[money_df['year']>2013].reset_index(drop=True)

#### 3. Number of branches

In [50]:
branch_df = pd.read_excel(r'/content/drive/MyDrive/ASU/CIS791_Chen_Final/uds-data/hc_site_zips.xlsx')

In [18]:
branch_df['Fiscal Year'] = branch_df['Fiscal Year'].replace(to_replace={'All Other Clinic Types':np.nan, 'School':np.nan, 'Health Center Location Setting':np.nan,
                                                                    'Unknown':np.nan, 'Hospital':np.nan, 'Nursing Home':np.nan, 'Domestic Violence':np.nan,
                                                                    'Correctional Facility':np.nan})
branch_df['Fiscal Year'] = branch_df['Fiscal Year'].fillna(method="ffill")
branch_df = branch_df[branch_df['Activity Code']!='H80'].reset_index(drop=True)
branch_df = branch_df.drop(columns=['Grantee Name'])
branch_df.columns = branch_df.iloc[0].values
branch_df = branch_df[branch_df['Health Center Type']!='Health Center Type']
branch_df = branch_df[branch_df['Grant #'].isin(grant_list)].reset_index(drop=True)
branch_df = branch_df.rename(columns={'2019':'year'})
branch_df = branch_df.groupby(['Grant #','year']).count()[['Site Name']].reset_index(level=[0,1])
branch_df.columns=['Grant Number', 'year', 'num_branches']
branch_df = branch_df[branch_df['year']!=2013].reset_index(drop=True)

In [169]:
branch_df

,Grant Number,year,num_branches
0,H80CS00001,2014,11
1,H80CS00001,2015,11
2,H80CS00001,2016,11
3,H80CS00001,2017,11
4,H80CS00001,2018,11
...,...,...,...
5652,H80CS33795,2020,12
5653,H80CS33884,2020,5
5654,H80CS33907,2020,4
5655,H80CS35351,2020,1


### Market characteristics: Get County information from HC Sites

In [91]:
# Clean grant data from website
## Follow year above
info_df = pd.read_excel(r'/content/drive/MyDrive/ASU/CIS791_Chen_Final/uds-data/hc_site_zips.xlsx')
info_df['Fiscal Year'] = info_df['Fiscal Year'].replace(to_replace={'All Other Clinic Types':np.nan, 'School':np.nan, 'Health Center Location Setting':np.nan,
                                                                    'Unknown':np.nan, 'Hospital':np.nan, 'Nursing Home':np.nan, 'Domestic Violence':np.nan,
                                                                    'Correctional Facility':np.nan})
info_df['Fiscal Year'] = info_df['Fiscal Year'].fillna(method="ffill")

In [92]:
# Clean data
df_1 = info_df[info_df['Program Name']=='Health Center Program (H80)'].reset_index(drop=True)
df_2 = info_df[info_df['Program Name']!='Health Center Program (H80)'].reset_index(drop=True)
df_2 = df_2.drop(columns=['Grantee Name'])
df_2.columns = df_2.iloc[0].values
df_2 = df_2[df_2['Services Delivered at Site']!='Services Delivered at Site'].reset_index(drop=True)
df_2 = df_2.rename(columns={'2019':'year', 'County':'county_site', 'State':'state_site', 'Grant #':'Grant Number', 'BHCMIS Org. ID':'BHCMIS ID', 
                            'ZIP':'zip_site', 'State County FIPS Code':'FIPS_site', 'HHS Region':'hhs_site', 'Rural Status':'rural_site'})
df_1 = df_1.rename(columns={'Grant #':'Grant Number', 'Fiscal Year':'year', 'State': 'state_center', 'County':'county_center', 
                            'ZIP':'zip_center', 'State County FIPS Code':'FIPS_center', 'HHS Region':'hhs_center', 'Rural Status':'rural_center'})
df_1['index_1'] = 'center'
df_2['index_2'] = 'site'

In [93]:
temp_df = pd.merge(df_1[['Grant Number', 'Grantee Name', 'year', 'state_center', 'county_center', 'zip_center', 'FIPS_center', 'hhs_center', 'rural_center', 'index_1']],
                   df_2[['Grant Number', 'Site Name', 'Health Center Type', 'Health Center Location Type', 'year', 'state_site', 'county_site', 'zip_site', 'FIPS_site', 'hhs_site', 'rural_site', 'index_2']],
                   on=['Grant Number', 'year'], how='outer')
temp_df = temp_df[temp_df['Grant Number'].isin(grant_list)].reset_index(drop=True) # should be our focus
# info_df = temp_df.copy()
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59071 entries, 0 to 59070
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Grant Number                 59071 non-null  object
 1   Grantee Name                 59071 non-null  object
 2   year                         59071 non-null  object
 3   state_center                 59071 non-null  object
 4   county_center                59071 non-null  object
 5   zip_center                   59071 non-null  object
 6   FIPS_center                  59071 non-null  object
 7   hhs_center                   59071 non-null  object
 8   rural_center                 59071 non-null  object
 9   index_1                      59071 non-null  object
 10  Site Name                    58828 non-null  object
 11  Health Center Type           58828 non-null  object
 12  Health Center Location Type  58828 non-null  object
 13  state_site                   58

In [94]:
# Centers that don't have sites should be center == site
site_df = temp_df[~temp_df['index_2'].isna()].reset_index(drop=True)
nosite_df = temp_df[temp_df['index_2'].isna()].reset_index(drop=True)
# fill blanks
nosite_df['Site Name'] = nosite_df['Grantee Name']
nosite_df['state_site'] = nosite_df['state_site']
nosite_df['county_site'] = nosite_df['county_center']
nosite_df['zip_site'] = nosite_df['zip_center']
nosite_df['FIPS_site'] = nosite_df['FIPS_center']
nosite_df['hhs_site'] = nosite_df['hhs_center']
nosite_df['rural_site'] = nosite_df['rural_center']
# Concat
info_df = pd.concat([site_df, nosite_df]).reset_index(drop=True)
# Fill missing values
## zip
info_df.loc[info_df['Site Name']=='GOLDEN HOUSE - Family Violence Center', ['zip_site']] = '54302'
info_df.loc[info_df['Site Name']=='The Julian Center Clinic', ['zip_site']] = '46202'
info_df.loc[info_df['Site Name']=='Westside Academy @ Blodgett - 312 Oswego St.', ['zip_site']] = '13204'
info_df.loc[info_df['Site Name']=='EDDY CENTER ADULT SHELTER', ['zip_site']] = '06457'
info_df.loc[info_df['Site Name']=='WYA AT CRYSALIS', ['zip_site']] = '06450'
# All other missinig are mostly domestic violence HCs
info_df = info_df[~info_df['zip_site'].isna()].reset_index(drop=True)
info_df['zip5_site'] = [int(str(i)[:5]) for i in info_df['zip_site']]

## county
info_df.loc[info_df['Site Name']=='GOLDEN HOUSE - Family Violence Center', ['FIPS_site']] = '55009'
info_df.loc[info_df['Site Name']=='The Julian Center Clinic', ['FIPS_site']] = '18097'
info_df.loc[info_df['Site Name']=='EDDY CENTER ADULT SHELTER', ['FIPS_site']] = '09007'
info_df.loc[info_df['Site Name']=='Plainfield Central School SBHC', ['FIPS_site']] = '09015'
info_df.loc[info_df['Site Name']=='WYA AT CRYSALIS', ['FIPS_site']] = '09009'
info_df.loc[info_df['Site Name']=='Westside Academy @ Blodgett - 312 Oswego St.', ['FIPS_site']] = '36067'
info_df.loc[info_df['Site Name']=='Santa Fe Suites Outreach Service Site', ['FIPS_site']] = '35049'

## state
info_df['state_site'] = info_df['state_site'].replace(to_replace={'XX':np.nan})
info_df['state_fips_site'] = [str(i)[:2] for i in info_df['FIPS_site']]
fips_df = info_df[['state_fips_site', 'state_site']].drop_duplicates().dropna().sort_values(by=['state_fips_site']).reset_index(drop=True)
info_df = pd.merge(info_df.drop(columns=['state_site']), fips_df, on=['state_fips_site'])

info_df = info_df[info_df['year']!=2013].reset_index(drop=True)

In [95]:
info_df.info() # 59002

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59027 entries, 0 to 59026
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Grant Number                 59027 non-null  object
 1   Grantee Name                 59027 non-null  object
 2   year                         59027 non-null  object
 3   state_center                 59027 non-null  object
 4   county_center                59027 non-null  object
 5   zip_center                   59027 non-null  object
 6   FIPS_center                  59027 non-null  object
 7   hhs_center                   59027 non-null  object
 8   rural_center                 59027 non-null  object
 9   index_1                      59027 non-null  object
 10  Site Name                    59027 non-null  object
 11  Health Center Type           58784 non-null  object
 12  Health Center Location Type  58784 non-null  object
 13  county_site                  59

In [96]:
# For proportion
def get_unique(dataframe, var):
  temp_gb = dataframe.groupby(['Grant Number', 'year']).nunique()[[var]].reset_index(level=[0,1]).rename(columns={var:'num_unique_{}'.format(var)})
  temp_gb['num_unique_{}'.format(var)] = 1/temp_gb['num_unique_{}'.format(var)]
  dataframe = pd.merge(dataframe, temp_gb, on=['Grant Number', 'year'])
  return dataframe

info_df = get_unique(dataframe = info_df, var='zip5_site')
info_df = get_unique(dataframe = info_df, var='FIPS_site')
info_df = get_unique(dataframe = info_df, var='hhs_site')
info_df = get_unique(dataframe = info_df, var='rural_site')

#### 1. Population

In [97]:
pop_df = pd.DataFrame()
for i in range(2014, 2021):
  print(i)
  temp_df = pd.read_csv('/content/drive/MyDrive/ASU/Telehealth/Data/Controls/ACSDP5Y{}.DP05.csv'.format(i))
  temp_df = temp_df.replace(to_replace={'(X)':np.nan})
  temp_df = temp_df[['GEO_ID','DP05_0001E']]
  temp_df['year'] = i
  pop_df = pop_df.append(temp_df)
  print("Done!")

pop_df['county_fips'] = [str(i)[-5:] for i in pop_df['GEO_ID']]
pop_df = pop_df[pop_df['GEO_ID']!='id'].reset_index(drop=True)
pop_df.columns = ['GEO_ID','county_population','year','county_fips']
pop_df = pop_df[['county_fips','year','county_population']]
pop_df['county_population'] = [float(i) for i in pop_df['county_population']]

2014


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,2,4,6,8,10,12,14,16,17,18,19,20,21,22,23,24,26,28,30,32,34,36,38,40,42,44,45,46,47,48,49,50,51,52,54,56,57,58,59,60,61,62,63,64,65,68,70,72,73,74,75,76,77,78,79,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,230,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,258,260,262,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,282,284,285,286,287,288,289,29

Done!
2015


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,2,4,6,8,10,12,14,16,17,18,19,20,21,22,23,24,26,28,30,32,34,36,38,40,42,44,45,46,47,48,49,50,51,52,54,56,57,58,59,60,61,62,63,64,65,68,70,72,73,74,75,76,77,78,79,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,230,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,258,260,262,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,282,284,285,286,287,288,289,29

Done!
2016
Done!
2017


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,2,4,6,8,10,12,16,18,20,21,22,23,24,25,26,27,28,30,32,34,36,38,40,42,44,46,48,49,50,51,52,53,54,55,56,58,60,61,62,63,64,65,66,67,68,69,72,74,76,77,78,79,80,82,84,85,86,87,88,89,90,91,92,94,96,98,100,102,104,106,108,112,114,116,118,120,122,128,130,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,250,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,278,280,282,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300

Done!
2018


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (5,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,127,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,155,157,159,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,179,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,221,222,223,225,226,227,228,229,230,231,232,234,236,238,240,242,244,246,250,252,254,255,256,257,258,259,260,261,262,264,266,268,270,272,274,276,278,280,282,283,284,285,286,287,288

Done!
2019
Done!
2020
Done!


In [98]:
pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22541 entries, 0 to 22540
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   county_fips        22541 non-null  object 
 1   year               22541 non-null  int64  
 2   county_population  22541 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 528.4+ KB


#### 2. Median household income

In [99]:
income_df = pd.DataFrame()
for i in range(2014, 2021):
  print(i)
  temp_df = pd.read_csv('/content/drive/MyDrive/ASU/Telehealth/Data/Controls/ACSST5Y{}.S1903.csv'.format(i))
  temp_df = temp_df.replace(to_replace={'(X)':np.nan})
  if i<2017:
    temp_df = temp_df[['GEO_ID','S1903_C02_001E']]
    temp_df.columns =['GEO_ID', 'median_household_income']
  elif i>=2017:
    temp_df = temp_df[['GEO_ID','S1903_C03_001E']]
    temp_df.columns =['GEO_ID', 'median_household_income']
    
  temp_df['year'] = i
  income_df = income_df.append(temp_df)
  print("Done!")

income_df['county_fips'] = [str(i)[-5:] for i in income_df['GEO_ID']]
income_df.columns = ['GEO_ID','median_household_income','year','county_fips']
income_df = income_df[income_df['GEO_ID']!='id'].reset_index(drop=True)
income_df = income_df[['county_fips','year','median_household_income']]
income_df['median_household_income'] = income_df['median_household_income'].replace(to_replace={'-':np.nan})

income_df.loc[(income_df['county_fips']=='48301')&(income_df['year']==2014), 'median_household_income'] = 48125
income_df.loc[(income_df['county_fips']=='35039')&(income_df['year']==2018), 'median_household_income'] = 36687 #(33422+39952)/2
income_df.loc[(income_df['county_fips']=='48243')&(income_df['year']==2020), 'median_household_income'] = 52982

income_df['median_household_income'] = [float(i) for i in income_df['median_household_income']]

2014
Done!
2015
Done!
2016
Done!
2017
Done!
2018
Done!
2019
Done!
2020
Done!


In [100]:
income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22541 entries, 0 to 22540
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   county_fips              22541 non-null  object 
 1   year                     22541 non-null  int64  
 2   median_household_income  22541 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 528.4+ KB


#### 3. Race

In [ ]:
race_df = pd.DataFrame()
for i in range(2014, 2021):
  print(i)
  temp_df = pd.read_csv('/content/drive/MyDrive/ASU/Telehealth/Data/Controls/ACSDP5Y{}.DP05.csv'.format(i))
  temp_df = temp_df.replace(to_replace={'(X)':np.nan})

  if i<=2016:
    temp_df = temp_df[['GEO_ID', 'DP05_0001E', 'DP05_0032E', 'DP05_0033E', 'DP05_0034E', 'DP05_0039E', 'DP05_0047E', 'DP05_0066E']]
    temp_df.columns = ['GEO_ID', 'county_population', 'county_population_white', 'county_population_black', 'county_population_native_american', 'county_population_asian', 'county_population_islander', 'county_population_hispanic']
  elif i>2016:
    temp_df = temp_df[['GEO_ID', 'DP05_0001E', 'DP05_0037E', 'DP05_0038E', 'DP05_0039E', 'DP05_0044E', 'DP05_0052E', 'DP05_0071E']]
    temp_df.columns = ['GEO_ID', 'county_population', 'county_population_white', 'county_population_black', 'county_population_native_american', 'county_population_asian', 'county_population_islander', 'county_population_hispanic']

  temp_df['year'] = i
  race_df = race_df.append(temp_df)
  print("Done!")

race_df['county_fips'] = [str(i)[-5:] for i in race_df['GEO_ID']]
race_df = race_df[race_df['GEO_ID']!='id'].reset_index(drop=True)
race_df = race_df.drop(columns=['GEO_ID'])
col_list = ['county_population', 'county_population_white', 'county_population_black', 'county_population_native_american', 'county_population_asian', 'county_population_islander', 'county_population_hispanic']
for col in col_list:
  race_df[col] = [float(i) for i in race_df[col]]

In [103]:
race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22541 entries, 0 to 22540
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   county_population                  22541 non-null  float64
 1   county_population_white            22541 non-null  float64
 2   county_population_black            22541 non-null  float64
 3   county_population_native_american  22541 non-null  float64
 4   county_population_asian            22541 non-null  float64
 5   county_population_islander         22541 non-null  float64
 6   county_population_hispanic         22541 non-null  float64
 7   year                               22541 non-null  int64  
 8   county_fips                        22541 non-null  object 
dtypes: float64(7), int64(1), object(1)
memory usage: 1.5+ MB


#### 4. Internet subscription

In [104]:
int_df = pd.DataFrame()
for i in range(2017, 2021):
  print(i)
  temp_df = pd.read_csv('/content/drive/MyDrive/ASU/Telehealth/Data/Controls/ACSST5Y{}.S2801.csv'.format(i))
  temp_df = temp_df.replace(to_replace={'(X)':np.nan})
  temp_df = temp_df[['GEO_ID', 'S2801_C01_001E', 'S2801_C01_002E', 'S2801_C01_011E', 'S2801_C01_012E', 'S2801_C01_019E']]
  temp_df['year'] = i
  int_df = int_df.append(temp_df)
  print("Done!")

int_df['county_fips'] = [str(i)[-5:] for i in int_df['GEO_ID']]
int_df = int_df[int_df['GEO_ID']!='id'].reset_index(drop=True)
int_df = int_df.rename(columns={'S2801_C01_001E':'num_households', 'S2801_C01_002E':'has_computer', 'S2801_C01_011E':'no_computer', 'S2801_C01_012E':'has_internet', 'S2801_C01_019E':'no_internet'})
int_df = int_df.drop(columns=['GEO_ID'])

int_df.loc[:,'num_households'] = int_df.loc[:,'num_households'].astype(int)
int_df.loc[:,'has_computer'] = int_df.loc[:,'has_computer'].astype(int)
int_df.loc[:,'no_computer'] = int_df.loc[:,'no_computer'].astype(int)
int_df.loc[:,'has_internet'] = int_df.loc[:,'has_internet'].astype(int)
int_df.loc[:,'no_internet'] = int_df.loc[:,'no_internet'].astype(int)

int_df['pct_computer'] = 100*(int_df['has_computer']/int_df['num_households'] )
int_df['pct_no_computer'] = 100*(int_df['no_computer']/int_df['num_households'] )
int_df['pct_internet'] = 100*(int_df['has_internet']/int_df['num_households'] )
int_df['pct_no_internet'] = 100*(int_df['no_internet']/int_df['num_households'] )

int_df = int_df[['county_fips','year','pct_computer','pct_no_computer','pct_internet','pct_no_internet']]

2017
Done!
2018
Done!
2019
Done!
2020
Done!


In [105]:
int_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12881 entries, 0 to 12880
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   county_fips      12881 non-null  object 
 1   year             12881 non-null  int64  
 2   pct_computer     12881 non-null  float64
 3   pct_no_computer  12881 non-null  float64
 4   pct_internet     12881 non-null  float64
 5   pct_no_internet  12881 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 603.9+ KB


#### 5. Language

In [ ]:
eng_df = pd.DataFrame()
for i in range(2014, 2021):
  print(i)
  
  temp_df = pd.read_csv('/content/drive/MyDrive/ASU/Telehealth/Data/Controls/ACSST5Y{}.S1601.csv'.format(i))
  temp_df = temp_df.replace(to_replace={'(X)':np.nan})
  if i==2014:
    temp_df = temp_df[['GEO_ID', 'S1601_C01_001E','S1601_C03_001E']]
    temp_df.columns = ['GEO_ID', 'population','pct_english_less']
  else:
    temp_df = temp_df[['GEO_ID', 'S1601_C01_001E','S1601_C05_001E']]
    temp_df.columns = ['GEO_ID', 'population','pct_english_less']
    
  temp_df['year'] = i
  eng_df = eng_df.append(temp_df)
  print("Done!")

eng_df['county_fips'] = [str(i)[-5:] for i in eng_df['GEO_ID']]
eng_df = eng_df[eng_df['GEO_ID']!='id'].reset_index(drop=True)
eng_df = eng_df.drop(columns=['GEO_ID'])

# 수치가 이상해서
eng1 = eng_df[eng_df['year']==2014]
eng2 = eng_df[eng_df['year']>2014]

eng2.loc[:, 'pct_english_less'] =  eng2.loc[:, 'pct_english_less'].astype(int)
eng2.loc[:, 'population'] =  eng2.loc[:, 'population'].astype(int)
eng2['pct_english_less'] = 100*(eng2['pct_english_less']/eng2['population'])

eng_df = pd.concat([eng1, eng2]).reset_index(drop=True)
eng_df = eng_df[['county_fips','year','pct_english_less']]

In [107]:
eng_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22541 entries, 0 to 22540
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   county_fips       22541 non-null  object
 1   year              22541 non-null  int64 
 2   pct_english_less  22541 non-null  object
dtypes: int64(1), object(2)
memory usage: 528.4+ KB


#### 6. Insurance

In [ ]:
insurance_df = pd.DataFrame()
for i in range(2014, 2021):
  print(i)
  temp_df = pd.read_csv('/content/drive/MyDrive/ASU/Telehealth/Data/Controls/ACSDT5Y{}.B27010.csv'.format(i))
  temp_df = temp_df.replace(to_replace={'(X)':np.nan})
  temp_df = temp_df[['GEO_ID', 'B27010_001E', 'B27010_017E', 'B27010_033E', 'B27010_050E', 'B27010_066E',
                     'B27010_006E',	'B27010_022E',	'B27010_038E',	'B27010_055E', 'B27010_007E', 'B27010_023E', 'B27010_039E']]
  # temp_df.columns = ['GEO_ID', 'population','pct_english_less']
  temp_df['year'] = i
  insurance_df = insurance_df.append(temp_df)
  print("Done!")

insurance_df['county_fips'] = [str(i)[-5:] for i in insurance_df['GEO_ID']]
insurance_df = insurance_df.rename(columns={'B27010_001E':'population', 'B27010_017E':'uninsured_0_18', 'B27010_033E':'uninsured_19_34', 
                                            'B27010_050E':'uninsured_35_64', 'B27010_066E':'uninsured_65', 'B27010_006E':'medicare_0_18',	
                                            'B27010_022E':'medicare_19_34',	'B27010_038E':'medicare_35_64',	'B27010_055E':'medicare_65',
                                            'B27010_007E':'medicaid_0_18', 'B27010_023E':'medicaid_19_34', 'B27010_039E':'medicaid_35_64'})

insurance_df = insurance_df[insurance_df['GEO_ID']!='id'].reset_index(drop=True)
insurance_df = insurance_df.drop(columns=['GEO_ID'])

col_list = ['population', 'uninsured_0_18', 'uninsured_19_34', 'uninsured_35_64', 
            'uninsured_65', 'medicare_0_18', 'medicare_19_34', 'medicare_35_64',
            'medicare_65', 'medicaid_0_18', 'medicaid_19_34', 'medicaid_35_64']
for col in col_list:
  insurance_df.loc[:,'{}'.format(col)] = insurance_df.loc[:,'{}'.format(col)].astype(int)

insurance_df['num_uninsured'] = insurance_df['uninsured_0_18']+insurance_df['uninsured_19_34']+insurance_df['uninsured_35_64']+insurance_df['uninsured_65']
insurance_df['num_medicare'] = insurance_df['medicare_0_18']+insurance_df['medicare_19_34']+insurance_df['medicare_35_64']+insurance_df['medicare_65']
insurance_df['num_medicaid'] = insurance_df['medicaid_0_18']+insurance_df['medicaid_19_34']+insurance_df['medicaid_35_64']

insurance_df['pct_uninsured'] = 100*(insurance_df['num_uninsured']/insurance_df['population'])
insurance_df['pct_medicaid'] = 100*(insurance_df['num_medicaid']/insurance_df['population'])
insurance_df['pct_medicare'] = 100*(insurance_df['num_medicare']/insurance_df['population'])

insurance_df = insurance_df[['county_fips','year','pct_uninsured','pct_medicare','pct_medicaid']]

In [144]:
insurance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22541 entries, 0 to 22540
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   county_fips    22541 non-null  object 
 1   year           22541 non-null  int64  
 2   pct_uninsured  22541 non-null  float64
 3   pct_medicare   22541 non-null  float64
 4   pct_medicaid   22541 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 880.6+ KB


### County

In [145]:
info_df.loc[:, 'year'] = info_df.loc[:, 'year'].astype(int)
info_df = info_df[info_df['year']!=2013].reset_index(drop=True)
temp_df = reduce(lambda x, y: pd.merge(x,y, on=['FIPS_site', 'year'], how='outer'), [info_df.rename(columns={'county_fips':'FIPS_site'}), 
                                                                                     pop_df.rename(columns={'county_fips':'FIPS_site'}), 
                                                                                     race_df.drop(columns=['county_population']).rename(columns={'county_fips':'FIPS_site'}),
                                                                                     int_df.rename(columns={'county_fips':'FIPS_site'}),
                                                                                     eng_df.rename(columns={'county_fips':'FIPS_site'}),
                                                                                     insurance_df.rename(columns={'county_fips':'FIPS_site'})])
temp_df = temp_df[~temp_df['Grant Number'].isna()].reset_index(drop=True)
info_df = temp_df.copy()

In [146]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52309 entries, 0 to 52308
Data columns (total 41 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Grant Number                       52309 non-null  object 
 1   Grantee Name                       52309 non-null  object 
 2   year                               52309 non-null  int64  
 3   state_center                       52309 non-null  object 
 4   county_center                      52309 non-null  object 
 5   zip_center                         52309 non-null  object 
 6   FIPS_center                        52309 non-null  object 
 7   hhs_center                         52309 non-null  object 
 8   rural_center                       52309 non-null  object 
 9   index_1                            52309 non-null  object 
 10  Site Name                          52309 non-null  object 
 11  Health Center Type                 52105 non-null  obj

In [147]:
county_df = info_df[['Grant Number', 'year', 'FIPS_site', 'num_unique_FIPS_site']].drop_duplicates().sort_values(by=['Grant Number', 'year', 'num_unique_FIPS_site', 'FIPS_site']).reset_index(drop=True)
county_df = county_df[county_df['year']!='2013'].reset_index(drop=True)

In [148]:
# Population
county_df.loc[:, 'year'] = county_df.loc[:, 'year'].astype(int)
pop_df.loc[:, 'year'] = pop_df.loc[:, 'year'].astype(int)

temp_df = pd.merge(county_df, pop_df.rename(columns={'county_fips':'FIPS_site'}), on=['year', 'FIPS_site'], how='outer')
temp_df = temp_df[~temp_df['Grant Number'].isna()].reset_index(drop=True)
temp_df['proportion_county_population'] = temp_df['num_unique_FIPS_site']*temp_df['county_population']

## Proportion
temp_df_gb_pop = temp_df.groupby(['Grant Number', 'year']).sum()[['proportion_county_population']].rename(columns={'proportion_county_population':'hc_population'}).reset_index(level=[0,1])

# Income
county_df.loc[:, 'year'] = county_df.loc[:, 'year'].astype(int)
income_df.loc[:, 'year'] = income_df.loc[:, 'year'].astype(int)

temp_df = pd.merge(county_df, income_df.rename(columns={'county_fips':'FIPS_site'}), on=['year', 'FIPS_site'], how='outer')
temp_df = temp_df[~temp_df['Grant Number'].isna()].reset_index(drop=True)
temp_df['proportion_income'] = temp_df['num_unique_FIPS_site']*temp_df['median_household_income']

## Proportion
temp_df_gb_inc = temp_df.groupby(['Grant Number', 'year']).sum()[['proportion_income']].rename(columns={'proportion_income':'hc_household_income'}).reset_index(level=[0,1])

# Race
county_df.loc[:, 'year'] = county_df.loc[:, 'year'].astype(int)
race_df.loc[:, 'year'] = race_df.loc[:, 'year'].astype(int)

temp_df = pd.merge(county_df, race_df.rename(columns={'county_fips':'FIPS_site'}), on=['year', 'FIPS_site'], how='outer')
temp_df = temp_df[~temp_df['Grant Number'].isna()].reset_index(drop=True)

temp_df['proportion_county_pop'] = temp_df['num_unique_FIPS_site']*temp_df['county_population']
temp_df['proportion_county_white'] = temp_df['num_unique_FIPS_site']*temp_df['county_population_white']
temp_df['proportion_county_black'] = temp_df['num_unique_FIPS_site']*temp_df['county_population_black']
temp_df['proportion_county_native_american'] = temp_df['num_unique_FIPS_site']*temp_df['county_population_native_american']
temp_df['proportion_county_asian'] = temp_df['num_unique_FIPS_site']*temp_df['county_population_asian']
temp_df['proportion_county_islander'] = temp_df['num_unique_FIPS_site']*temp_df['county_population_islander']
temp_df['proportion_county_hispanic'] = temp_df['num_unique_FIPS_site']*temp_df['county_population_hispanic']

# ## Proportion
temp_df_gb_race = temp_df.groupby(['Grant Number', 'year']).sum()[['proportion_county_white','proportion_county_black','proportion_county_native_american','proportion_county_asian',
                                                                   'proportion_county_islander','proportion_county_hispanic','proportion_county_pop']].rename(columns={'proportion_county_population':'hc_population'}).reset_index(level=[0,1])

temp_df_gb_race['proportion_county_white'] = 100*(temp_df_gb_race['proportion_county_white']/temp_df_gb_race['proportion_county_pop'])
temp_df_gb_race['proportion_county_black'] = 100*(temp_df_gb_race['proportion_county_black']/temp_df_gb_race['proportion_county_pop'])
temp_df_gb_race['proportion_county_native_american'] = 100*(temp_df_gb_race['proportion_county_native_american']/temp_df_gb_race['proportion_county_pop'])
temp_df_gb_race['proportion_county_asian'] = 100*(temp_df_gb_race['proportion_county_asian']/temp_df_gb_race['proportion_county_pop'])
temp_df_gb_race['proportion_county_islander'] = 100*(temp_df_gb_race['proportion_county_islander']/temp_df_gb_race['proportion_county_pop'])
temp_df_gb_race['proportion_county_hispanic'] = 100*(temp_df_gb_race['proportion_county_hispanic']/temp_df_gb_race['proportion_county_pop'])

temp_df_gb_race.columns=['Grant Number', 'year', 'hc_white', 'hc_black', 'hc_native_american', 'hc_asian', 'hc_islander', 'hc_hispanic', 'hc_pop']
temp_df_gb_race = temp_df_gb_race.drop(columns=['hc_pop'])

# Internet
county_df.loc[:, 'year'] = county_df.loc[:, 'year'].astype(int)
int_df.loc[:, 'year'] = int_df.loc[:, 'year'].astype(int)

temp_df = pd.merge(county_df, int_df.rename(columns={'county_fips':'FIPS_site'}), on=['year', 'FIPS_site'], how='outer')
temp_df = temp_df[~temp_df['Grant Number'].isna()].reset_index(drop=True)

temp_df['proportion_county_pct_computer'] = temp_df['num_unique_FIPS_site']*temp_df['pct_computer']
temp_df['proportion_county_pct_no_computer'] = temp_df['num_unique_FIPS_site']*temp_df['pct_no_computer']
temp_df['proportion_county_pct_internet'] = temp_df['num_unique_FIPS_site']*temp_df['pct_internet']
temp_df['proportion_county_pct_no_internet'] = temp_df['num_unique_FIPS_site']*temp_df['pct_no_internet']

# ## Proportion
temp_df_gb_int = temp_df.groupby(['Grant Number', 'year']).sum()[['proportion_county_pct_computer','proportion_county_pct_no_computer','proportion_county_pct_internet','proportion_county_pct_no_internet']].reset_index(level=[0,1])
temp_df_gb_int = temp_df_gb_int[temp_df_gb_int['year']>2016].reset_index(drop=True)
temp_df_gb_int.colums=['Grant Number', 'year','hc_pct_computer','hc_pct_no_computer','hc_pct_internet','hc_pct_no_internet']

# Language
county_df.loc[:, 'year'] = county_df.loc[:, 'year'].astype(int)
eng_df.loc[:, 'year'] = eng_df.loc[:, 'year'].astype(int)
eng_df.loc[:,'pct_english_less'] = eng_df.loc[:,'pct_english_less'].astype(float)

temp_df = pd.merge(county_df, eng_df.rename(columns={'county_fips':'FIPS_site'}), on=['year', 'FIPS_site'], how='outer')
temp_df = temp_df[~temp_df['Grant Number'].isna()].reset_index(drop=True)

temp_df['proportion_county_pct_english_less'] = temp_df['num_unique_FIPS_site']*temp_df['pct_english_less']

# ## Proportion
temp_df_gb_eng = temp_df.groupby(['Grant Number', 'year']).sum()[['proportion_county_pct_english_less']].reset_index(level=[0,1])
temp_df_gb_eng.colums=['Grant Number', 'year','hc_pct_less_eng']


# Insurance
county_df.loc[:, 'year'] = county_df.loc[:, 'year'].astype(int)
insurance_df.loc[:, 'year'] = insurance_df.loc[:, 'year'].astype(int)

temp_df = pd.merge(county_df, insurance_df.rename(columns={'county_fips':'FIPS_site'}), on=['year', 'FIPS_site'], how='outer')
temp_df = temp_df[~temp_df['Grant Number'].isna()].reset_index(drop=True)

temp_df['proportion_county_pct_uninsured'] = temp_df['num_unique_FIPS_site']*temp_df['pct_uninsured']
temp_df['proportion_county_pct_medicare'] = temp_df['num_unique_FIPS_site']*temp_df['pct_medicare']
temp_df['proportion_county_pct_medicaid'] = temp_df['num_unique_FIPS_site']*temp_df['pct_medicaid']

# ## Proportion
temp_df_gb_insurance = temp_df.groupby(['Grant Number', 'year']).sum()[['proportion_county_pct_uninsured','proportion_county_pct_medicare','proportion_county_pct_medicaid']].reset_index(level=[0,1])
temp_df_gb_insurance.colums=['Grant Number', 'year','hc_pct_uninsuredd','hc_pct_medicare','hc_pct_medicaid']

# # ## Merge all
temp_df = reduce(lambda x, y: pd.merge(x, y, on=['Grant Number', 'year'], how='outer'),[temp_df_gb_pop, temp_df_gb_inc, temp_df_gb_race,
                                                                                        temp_df_gb_int,temp_df_gb_eng,temp_df_gb_insurance])
ctrl_df = temp_df.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


### Merge all

In [168]:
## Merge all
t4_df.loc[:, 'year'] = t4_df.loc[:, 'year'].astype(int)
money_df.loc[:, 'year'] = money_df.loc[:, 'year'].astype(int)
branch_df.loc[:, 'year'] = branch_df.loc[:, 'year'].astype(int)
ctrl_df.loc[:, 'year'] = ctrl_df.loc[:, 'year'].astype(int)

temp_df = reduce(lambda x, y: pd.merge(x, y, on=['Grant Number', 'year'], how='outer'), [t4_df, money_df, branch_df, ctrl_df])
temp_df = temp_df.rename(columns={'Financial Assistance':'grant_amount'})
temp_df = temp_df[temp_df['Grant Number'].isin(grant_list)].reset_index(drop=True)

In [176]:
temp_df.to_csv(rootPath+'ctrl_df.csv', index=False)